In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

c:\Users\mrmoh\anaconda3\envs\Nerf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
torch.zeros((160000, 100, 3))

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [7]:
bb = torch.zeros((160000, 100, 3))
bb.shape

torch.Size([160000, 100, 3])

In [8]:
H = 400
W = 400
f = 1200


In [9]:
rays_o = np.zeros((H*W, 3))
rays_d = np.zeros((H*W, 3))

u = np.arange(W)
v = np.arange(H)
u, v = np.meshgrid(u, v)

dirs = np.stack((u - W / 2,
                 -(v - H / 2),
                 - np.ones_like(u) * f), axis=-1)
rays_d = dirs / np.linalg.norm(dirs, axis=-1, keepdims=True)
rays_d = rays_d.reshape(-1, 3)

In [10]:
def plot_rays(o, d, t):
    
    fig = plt.figure(figsize=(12, 12))
    ax = fig.gca(projection='3d')
    
    pt1 = o
    pt2 = o + t * d
    
    for p1, p2 in zip(pt1[::100], pt2[::100]):
        plt.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]])
    
    
    plt.show()

In [11]:
class Sphere():
    
    def __init__(self, p, r, c):
        self.p = p
        self.r = r
        self.c = c
        
    def intersect(self, x):
        
        # (x- xc)^2 + (y-yc)^2 + (z-zc)^2 <= r^2 (this is the main equation)
        # (x- xc)^2 + (y-yc)^2 + (z-zc)^2 = r^2 - eqn of sphere but to get inside we give <.

        learn = x[:, 0]**2 + x[:, 1]**2 + x[:, 2]**2 < self.r**2
        #here x[:, 0] is x component, x[:, 1] is y component, x[:, 2] is z component of the ray
        # here r**2 is the radius of the sphere in the x, y, z direction
        # so if x[:, 0]**2 + x[:, 1]**2 + x[:, 2]**2 < self.r**2 is the condition for the ray to intersect the sphere or to be inside the sphere
        # then the color of the ray will be the color ofthe sphere 

        # cond = x[:, 0]**2 + x[:,1]**2 + x[:, 2]**2 < self.r**2
        #so any point that satisfies this condition is inside the sphere 
        cond = (x[:, 0] - self.p[0])**2 + (x[:, 1] - self.p[1])**2 + (x[:, 2] - self.p[2])**2 <= self.r**2
        #taking in account the center of the sphere x0, y0, z0 - self.p[0], self.p[1], self.p[2]
        num_rays = o.shape[0] # this is the number of rays
        colors = np.zeros((num_rays, 3))
        density = np.zeros((num_rays, 1)) # 1 cuz we only have one value for the density         
        
        colors[cond] = self.c # giving the ray the color of the sphere
        density[cond] = 10 # giving the ray the density of the sphere
        
        return colors

In [12]:
s = Sphere(np.array([0., 0., -1.]), .1, np.array([1., 0., 0.]))

In [13]:
# def rendering(reays_o, rays_d, tn, tf):

#     colors = np.zeros((reays_o.shape[0], 3))
#     density = np.zeros((reays_o.shape[0], 1))

#     for i in range(reays_o.shape[0]):
#         colors[i] = s.intersect(reays_o[i])
#         density[i] = 10

#     return colors, density